In [12]:
import requests
from bs4 import BeautifulSoup
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [73]:
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0') 
    #find_element_by_css_selector 함수를 사용해 요소 찾기
    first.click()
    time.sleep(3) #로딩을 위해 3초 대기
#본문 내용, 작성 일시, 위치 정보 및 해시태그(#) 추출
import re 
def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')    
    # 2. 본문 내용 가져오기
    try:  			#여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text 
        			#첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    # 4. 작성 일자 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1] 
    except:
        like = 0
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    data = [content, date, like, place, tags]
    return data 

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()
    time.sleep(3)

In [74]:
driver = webdriver.Chrome("chromedriver.exe") # Chromedriver PATH
driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()


username = '여기다 인스타 아이디 적어'
password = '비밀번호'

 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)
 
time.sleep(3)

driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

time.sleep(3)


In [75]:
word = '기분'

driver.get('https://www.instagram.com/explore/tags/' + word)

time.sleep(3)

In [76]:
select_first(driver) 
#5. 비어있는 변수(results) 만들기
results = [] 
#여러 게시물 크롤링하기
target = 1000 #크롤링할 게시물 수
for i in range(target):
    data = get_content(driver) #게시물 정보 가져오기
    results.append(data)
    move_next(driver)    
print(results[:2])

[['술냉장고가 채워지고 있습니다. 제주에서도 프리미엄급 사케를 공급하기위해 아는 인맥 총 동원중입니다~기분을 더욱 기분스럽게~술에 최선을 다해보겠습니다.#사카쇼#酒匠#사케마스터#이자까야기분#이자카야#이자카야기분#기분#일본소주#잔술#사케#와규#이자까야#쇼추#이모쇼추#居酒屋#居酒屋気分#気分#사케스쿨#반포동술집#焼酎#日本酒#さけ#제주기분#건입동기분#술입고#나홍규대리님감사', '2020-12-29', '113', '', ['#사카쇼', '#酒匠', '#사케마스터', '#이자까야기분', '#이자카야', '#이자카야기분', '#기분', '#일본소주', '#잔술', '#사케', '#와규', '#이자까야', '#쇼추', '#이모쇼추', '#居酒屋', '#居酒屋気分', '#気分', '#사케스쿨', '#반포동술집', '#焼酎', '#日本酒', '#さけ', '#제주기분', '#건입동기분', '#술입고', '#나홍규대리님감사']], ['🖋소중한 사람들을 잃지 않으려면 지켜야 할 것.자신의 기분에 따라 상대방을 대하는 사람들을 몇몇 봤다. 기분이 좋을 때는 문제가 되지 않는데, 기분이 안 좋을 때가 문제였다. 표정이며 말투며 온몸으로 안 좋은 기분을 표출했으니까. 주변 사람들이 무척이나 힘들어하는 것을 많이 봤다. 처음에는 다들 어르고 달래며 그들에게 맞춰주다가도 결국에는 참지 못하고 그들을 떠났다.대부분의 사람들은 자신의 기분을 완벽히 숨기지는 못한다. 하지만 완벽히 숨기지는 못해도 강하게 표출하지는 말아야 한다. 자신의 기분 때문에 상대방의 기분까지 망쳐 놓아서는 안 된다.📔도서 <나의 청춘에게>', '2021-01-14', '1,734', '', []]]


In [80]:
df = pd.DataFrame(results,columns=['content','date','like',' ','tags'])
df

,content,date,like,,tags
0,술냉장고가 채워지고 있습니다. 제주에서도 프리미엄급 사케를 공급하기위해 아는 인맥 ...,2020-12-29,113,,"[#사카쇼, #酒匠, #사케마스터, #이자까야기분, #이자카야, #이자카야기분, #..."
1,🖋소중한 사람들을 잃지 않으려면 지켜야 ...,2021-01-14,"1,734",,[]
2,나의 바다를 사랑하는 법 🐳⠀날씨가 추워지니...,2020-12-15,"2,356",,"[#더블유의소소생각, #매일맑지않아도, #하..."
3,인생에서 원하는 것을 얻기 위한 첫번째 단계는내가 무엇을 원하는지 결정하는 것이다-...,2021-01-08,"2,922",,"[#더블유의소소생각, #인생은예측불허, #그리하여생은의미를갖는다]"
4,고마우니까 고마운 너에게 💭⠀수많은 빛나는 ...,2020-12-21,"2,876",,"[#더블유의소소생각, #너는내게충분히고마ᄋ..."
...,...,...,...,...,...
995,억울...#스트레스 #속상 #억울 #기분,2021-01-09,5,,"[#스트레스, #속상, #억울, #기분]"
996,오늘 아이폰 12 프로로 바꿨다!!! 이젠 나도 아이폰 유저다!!! 이제 아이폰 공...,2021-01-09,13,,"[#아이폰, #폰바꿈, #기분, #조아]"
997,"영원히 살 것처럼 꿈을꾸고,내일 죽을 ...",2021-01-09,62,,"[#오늘의훈녀, #얼스타그램, #셀스타그램, ..."
998,#짧은글 #짧은시 #시 #글귀 #글 #에세이 #생각 #마음 #기분 #감성글 #자작시...,2021-01-09,64,,"[#짧은글, #짧은시, #시, #글귀, #글, #에세이, #생각, #마음, #기분,..."
